

# Import libraries.


In [ ]:
# Core
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import random
import os
import cv2
#tesor fow & keras
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2     
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense,BatchNormalization,Dropout,Input
from keras.models import Sequential, Model
from keras.layers import Conv2D,GlobalMaxPooling2D
from tensorflow.keras.applications import  Xception,VGG16,InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#cnn
from tensorflow.keras import datasets, layers, models

from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.engine.base_layer import Layer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
seed = 42
np.random.seed =seed

In [ ]:
image_path = '../input/happy-whale-and-dolphin/'


# Bussiness Task
Developing  a model to match individual whales and dolphins by unique—but often subtle—characteristics of their natural markings. 

I hope you find this NoteBook helpful and some <span style="color:red;">**UPVOTES**</span> would be appreciated.




# Data Preperation.
1.   read csv file to get all data about train image 
2.   Check Null data 
3.   clear duplicate data





In [ ]:
train_df = pd.read_csv(image_path+'train.csv')
train_df.head(10)

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.isnull().sum().sort_values(ascending=False)

In [ ]:
train_df.drop_duplicates(inplace=True)

In [ ]:
cols =train_df.columns
colours = ['#4624b8', '#d080b6'] # specify the orange  - yellow is missing. blue is not missing.
sns.heatmap(train_df[cols].isnull(), cmap=sns.color_palette(colours))

**Get Unique data for each column in  Data Frame** 

In [ ]:
sorted(train_df['species'].unique())

From above we found that there are some coliumn name need to be corrected.

In [ ]:
train_df['species'].replace({
                          "pilot_whale": "short_finned_pilot_whale",
                          "globis": "short_finned_pilot_whale",
                          "bottlenose_dolpin": "bottlenose_dolphin",
                          "kiler_whale": "killer_whale"}, inplace=True)


In [ ]:
total_train =train_df['species'].count()
totalspecies=train_df['species'].nunique()
totalindividual_id= train_df['individual_id'].nunique()
print(f'Total train images         : {total_train}')
print(f'Total train species        : {totalspecies}')
print(f'Total train individualId   : {totalindividual_id}')

In [ ]:
labels = list(train_df['individual_id'].unique())

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='species',data=train_df,palette='flare')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import plotly.express as px
fig = px.sunburst(train_df,path=['species'])
fig.show()

# Whale analysis and Visualization 🐳
1.  collect all types of whales 
2.  collect all walles data  
3.  whale statistics 
4.  whales Visualize 

In [ ]:
#  all whales types
allwales = [data for data in train_df['species'].unique()  if not "dolphin" in data and  not "dolpin" in data]
#get all data for each  type
whales_df = train_df[train_df['species'].isin(allwales)]
whales_df.head(10)

In [ ]:
sns.countplot(x='species',data=whales_df,palette='flare')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import plotly.express as px
fig = px.sunburst(whales_df,path=['species'])
fig.show()

**whale statistics**

In [ ]:
total = len(allwales)
print(f' total train whale is         :    {total}')
whale_images = whales_df['image'].nunique()
print(f' train whale images  is       : {whale_images}')
whale_ids=whales_df['individual_id'].nunique()
print(f' train whale individualId  is : {whale_ids}')

Notes 
 

*   humpback_whale & beluga has most count of image in data set 
*   pygemy_killer_wall has low image in whale data set
*  there are variation of image of whaels between its types.



# Dolphin analysis and Visualization 🐬
1.  collect all types of Dolphin 
2.  collect all Dolphin data  
3.  Dolphin statistics 
4.  Dolphin Visualize 

In [ ]:
# get all dolphin types
all_dolphine = [data for data in train_df['species'].unique()  if  "dolphin" in data or  "dolpin" in data]
#get all data for each specific type
dolphin_df = train_df[train_df['species'].isin(all_dolphine)]
dolphin_df.head(10)

In [ ]:
sns.countplot(x='species',data=dolphin_df,palette='flare')
plt.xticks(rotation=90)
plt.show()

In [ ]:
import plotly.express as px
fig = px.sunburst(dolphin_df,path=['species'])
fig.show()

In [ ]:
total = len(all_dolphine)
print(f' total train Dolphin is         :    {total}')
dolphin_images = dolphin_df['image'].nunique()
print(f' train dolphin images  is       : {dolphin_images}')
whale_ids=dolphin_df['individual_id'].nunique()
print(f' train dolphin individualId  is : {whale_ids}')

Notes 
 

*   humpback_whale bottlenose_dolphin of image in data set .
*   images of dolphine types  is very low which mean that it is difficult to diffreniate between dopline types so we need to increase its images 



# loading images  🐬vs 🐳

In [ ]:

plt.figure(figsize=(20,15))
plt.suptitle("Train Images", fontsize=20)
path =image_path+'/'+'train_images/'
counter =0
for i,img in enumerate(train_df['image'])  :
        plt.subplot(5,7,i+1)
        full_image= plt.imread(path+img)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.xlabel(train_df['species'][i])
        plt.imshow(full_image, cmap=plt.cm.binary) 
        if i == 34:
            break

# Buliding CNN Model

**i will use Data augmentation,as it  is a set of techniques to artificially increase the amount of data by generating new data points from existing data. This includes making small changes to data or using deep learning models to generate new data points.** 



In [ ]:
batch_size = 64
#create image generator for images 
image_gen = ImageDataGenerator(
                                 rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.5,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  fill_mode='nearest',
                                   horizontal_flip=True,
                                   rotation_range = 20,
                               validation_split=0.2 
                               )

# Create Image Data Generator for Test/Validation Set
test_data_gen = ImageDataGenerator(  
                                   rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.5,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  fill_mode='nearest',
                                   horizontal_flip=True,
                                   rotation_range = 20,
                                   validation_split=0.2)        
train = image_gen.flow_from_dataframe(
      train_df,
      path,
      x_col='image',
      y_col='individual_id',
      target_size=(64,64),
      class_mode='categorical',
      shuffle=True, 
      batch_size=batch_size,
      labels =labels,
      subset = "training"
      )
validate = test_data_gen.flow_from_dataframe(
      train_df,
      path,
      x_col='image',
      y_col='individual_id',
      target_size=(64,64),
      class_mode='categorical',
      shuffle=True, 
      batch_size=batch_size,
      subset = "validation",
      labels =labels
      )


In [ ]:
# get shape of train data 
for train_img , train_label in train :
    print('image shape ',train_img.shape)
    print('label  shape ',train_label.shape)
    break 

In [ ]:
# get shape of train data 
for v_img , v_label in validate :
    print('image shape ',v_img.shape)
    print('label  shape ',v_label.shape)
    break 

In [ ]:
cnn_model = Sequential()
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(64,(3,3),padding ='Same',activation = 'relu',input_shape=(64,64,3)))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Conv2D(64,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Conv2D(128,(3,3),padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(layers.Conv2D(128,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(layers.Conv2D(256,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(BatchNormalization())
cnn_model.add(layers.Conv2D(256,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(BatchNormalization())
cnn_model.summary()
# cnn_model.add(Dropout(0.2))


In [ ]:

cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(1024, activation='relu'))
cnn_model.add(BatchNormalization())
# cnn_model.add(Dropout(0.7))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(BatchNormalization())
# cnn_model.add(Dropout(0.3))
cnn_model.add(layers.Dense(15587, activation ='softmax'))
cnn_model.summary()


In [ ]:
cnn_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)

  **1-Defining Callbacks**

*   A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc)


**2-Reduce Learning Rate on Plateau**
*   Is used to reduce the learning rate when a metric has stopped improving.



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor="loss", mode="min",min_delta = 0,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [ early, learning_rate_reduction]

In [ ]:
# Training model
n_training_samples = len(train)
n_validation_samples = len(validate)
history = cnn_model.fit(
    train,
    epochs=100,
    validation_data=validate,
    validation_steps=n_validation_samples//batch_size,
    steps_per_epoch =n_training_samples//batch_size,
    shuffle = True,
    callbacks=callbacks_list
    )


Thanks for   <span style="color:red;">**Reviewing**</span>   My kernel also, if you have any comments or suggestion please raise it